# Day 6: Permutation Importance & Partial Dependence

## Learning Objectives
- Understand permutation importance vs built-in importance
- Create Partial Dependence Plots (PDPs)
- Individual Conditional Expectation (ICE) plots

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.inspection import permutation_importance, partial_dependence
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("✅ Libraries loaded!")
print("📚 Day 6: Permutation Importance & Partial Dependence")

In [ ]:
# ============================================================
# PREPARE DATA
# ============================================================

print("PREPARING DATA")
print("="*60)

np.random.seed(42)
n_days = 1500

returns = np.random.normal(0.0002, 0.015, n_days)
for i in range(1, len(returns)):
    returns[i] += 0.05 * returns[i-1]

prices = 100 * np.cumprod(1 + returns)
df = pd.DataFrame({'price': prices, 'returns': returns})

# Features
df['ret_5d'] = df['price'].pct_change(5)
df['ret_20d'] = df['price'].pct_change(20)
df['vol_5d'] = df['returns'].rolling(5).std()
df['vol_20d'] = df['returns'].rolling(20).std()
df['mom_5d'] = df['returns'].rolling(5).sum()
df['mom_20d'] = df['returns'].rolling(20).sum()

gains = df['returns'].where(df['returns'] > 0, 0).rolling(14).mean()
losses = (-df['returns'].where(df['returns'] < 0, 0)).rolling(14).mean()
df['rsi'] = 100 - (100 / (1 + gains / (losses + 1e-10)))

df['vol_ratio'] = df['vol_5d'] / df['vol_20d']

df['target'] = (df['returns'].shift(-1) > 0).astype(int)
df = df.dropna()

feature_cols = ['ret_5d', 'ret_20d', 'vol_5d', 'vol_20d', 'mom_5d', 'mom_20d', 'rsi', 'vol_ratio']
X = df[feature_cols].values
y = df['target'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

split = int(len(X) * 0.7)
X_train, X_test = X_scaled[:split], X_scaled[split:]
y_train, y_test = y[:split], y[split:]

print(f"Features: {feature_cols}")

## Part 1: Permutation Importance

In [ ]:
# ============================================================
# TRAIN MODEL & PERMUTATION IMPORTANCE
# ============================================================

print("PERMUTATION IMPORTANCE")
print("="*60)

model = GradientBoostingClassifier(n_estimators=100, max_depth=4, random_state=42)
model.fit(X_train, y_train)

print(f"Test accuracy: {model.score(X_test, y_test):.3f}")

# Built-in importance
builtin_imp = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

# Permutation importance (on test set)
perm_imp = permutation_importance(model, X_test, y_test, n_repeats=30, random_state=42)

perm_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': perm_imp.importances_mean,
    'std': perm_imp.importances_std
}).sort_values('importance', ascending=False)

print("\nBuilt-in Importance:")
print(builtin_imp.head(5).to_string(index=False))

print("\nPermutation Importance (test set):")
print(perm_df.head(5).to_string(index=False))

In [ ]:
# ============================================================
# COMPARE IMPORTANCE METHODS
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

builtin_imp.plot(x='feature', y='importance', kind='barh', ax=axes[0],
                 color='steelblue', alpha=0.7, legend=False)
axes[0].set_xlabel('Importance')
axes[0].set_title('Built-in (Gini) Importance', fontweight='bold')
axes[0].invert_yaxis()

perm_df.plot(x='feature', y='importance', kind='barh', ax=axes[1],
             xerr=perm_df['std'], color='coral', alpha=0.7, legend=False)
axes[1].set_xlabel('Mean Accuracy Decrease')
axes[1].set_title('Permutation Importance (with std)', fontweight='bold')
axes[1].axvline(0, color='gray', linestyle='--')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print("\n💡 KEY INSIGHT: Permutation importance is more reliable because:")
print("   - It measures actual model performance drop")
print("   - It's computed on test data (no training bias)")
print("   - Built-in importance can be biased toward high-cardinality features")

## Part 2: Partial Dependence Plots

In [ ]:
# ============================================================
# PARTIAL DEPENDENCE PLOTS
# ============================================================

print("PARTIAL DEPENDENCE PLOTS")
print("="*60)

def plot_partial_dependence(model, X, feature_idx, feature_name, ax, n_points=50):
    """
    Create a partial dependence plot.
    
    PDP shows the marginal effect of a feature on the prediction:
    Average prediction as we vary one feature, keeping others at their actual values.
    """
    # Get range for feature
    feat_vals = np.linspace(X[:, feature_idx].min(), X[:, feature_idx].max(), n_points)
    
    # Calculate PD
    pdp_values = []
    for val in feat_vals:
        X_temp = X.copy()
        X_temp[:, feature_idx] = val
        avg_pred = model.predict_proba(X_temp)[:, 1].mean()
        pdp_values.append(avg_pred)
    
    ax.plot(feat_vals, pdp_values, color='steelblue', linewidth=2)
    ax.set_xlabel(feature_name)
    ax.set_ylabel('Partial Dependence')
    ax.set_title(f'PDP: {feature_name}', fontweight='bold')
    ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5)
    ax.grid(True, alpha=0.3)
    
    # Add feature histogram
    ax2 = ax.twinx()
    ax2.hist(X[:, feature_idx], bins=30, alpha=0.2, color='gray')
    ax2.set_ylabel('Frequency', color='gray')
    ax2.tick_params(axis='y', labelcolor='gray')

# Plot PDPs for top features
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

top_features = perm_df.head(4)['feature'].tolist()
for ax, feat in zip(axes.flat, top_features):
    feat_idx = feature_cols.index(feat)
    plot_partial_dependence(model, X_test, feat_idx, feat, ax)

plt.tight_layout()
plt.show()

## Part 3: ICE Plots (Individual Conditional Expectation)

In [ ]:
# ============================================================
# ICE PLOTS
# ============================================================

print("ICE PLOTS")
print("="*60)

def plot_ice(model, X, feature_idx, feature_name, ax, n_samples=50, n_points=50):
    """
    Create an ICE plot with centered lines.
    
    ICE shows individual prediction trajectories as we vary a feature.
    It reveals heterogeneous effects that PDP averages over.
    """
    feat_vals = np.linspace(X[:, feature_idx].min(), X[:, feature_idx].max(), n_points)
    
    # Random subset of samples
    sample_idx = np.random.choice(len(X), min(n_samples, len(X)), replace=False)
    
    ice_curves = []
    for i in sample_idx:
        curve = []
        for val in feat_vals:
            X_temp = X[i:i+1].copy()
            X_temp[0, feature_idx] = val
            pred = model.predict_proba(X_temp)[0, 1]
            curve.append(pred)
        ice_curves.append(curve)
    
    ice_curves = np.array(ice_curves)
    
    # Plot individual curves
    for curve in ice_curves:
        ax.plot(feat_vals, curve, color='steelblue', alpha=0.1, linewidth=0.5)
    
    # Plot mean (PDP)
    ax.plot(feat_vals, ice_curves.mean(axis=0), color='red', linewidth=2, label='Mean (PDP)')
    
    ax.set_xlabel(feature_name)
    ax.set_ylabel('Prediction Probability')
    ax.set_title(f'ICE Plot: {feature_name}', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Plot ICE for top features
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, feat in zip(axes.flat, top_features):
    feat_idx = feature_cols.index(feat)
    plot_ice(model, X_test, feat_idx, feat, ax)

plt.tight_layout()
plt.show()

print("\n💡 ICE reveals heterogeneous effects:")
print("   - If lines are parallel: homogeneous effect (PDP is representative)")
print("   - If lines cross: interaction effects exist (PDP may be misleading)")

## Part 4: 2D Partial Dependence (Interactions)

In [ ]:
# ============================================================
# 2D PARTIAL DEPENDENCE
# ============================================================

print("2D PARTIAL DEPENDENCE (INTERACTIONS)")
print("="*60)

def plot_2d_pdp(model, X, feat_idx1, feat_idx2, feat_name1, feat_name2, ax, n_points=20):
    """Create 2D PDP to show feature interactions."""
    
    vals1 = np.linspace(X[:, feat_idx1].min(), X[:, feat_idx1].max(), n_points)
    vals2 = np.linspace(X[:, feat_idx2].min(), X[:, feat_idx2].max(), n_points)
    
    pdp_2d = np.zeros((n_points, n_points))
    
    for i, v1 in enumerate(vals1):
        for j, v2 in enumerate(vals2):
            X_temp = X.copy()
            X_temp[:, feat_idx1] = v1
            X_temp[:, feat_idx2] = v2
            pdp_2d[i, j] = model.predict_proba(X_temp)[:, 1].mean()
    
    im = ax.contourf(vals2, vals1, pdp_2d, levels=20, cmap='RdYlBu_r')
    ax.set_xlabel(feat_name2)
    ax.set_ylabel(feat_name1)
    ax.set_title(f'2D PDP: {feat_name1} vs {feat_name2}', fontweight='bold')
    plt.colorbar(im, ax=ax)

# Plot 2D PDP for feature pairs
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# vol_5d vs mom_5d
idx1 = feature_cols.index('vol_5d')
idx2 = feature_cols.index('mom_5d')
plot_2d_pdp(model, X_test, idx1, idx2, 'vol_5d', 'mom_5d', axes[0])

# rsi vs ret_5d
idx1 = feature_cols.index('rsi')
idx2 = feature_cols.index('ret_5d')
plot_2d_pdp(model, X_test, idx1, idx2, 'rsi', 'ret_5d', axes[1])

plt.tight_layout()
plt.show()

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║      DAY 6 COMPLETE: PERMUTATION & PARTIAL DEPENDENCE            ║
╠══════════════════════════════════════════════════════════════════╣
║  ✓ Permutation importance (model-agnostic)                      ║
║  ✓ Comparison with built-in importance                          ║
║  ✓ Partial Dependence Plots (PDPs)                              ║
║  ✓ ICE plots for heterogeneous effects                          ║
║  ✓ 2D PDPs for feature interactions                             ║
╚══════════════════════════════════════════════════════════════════╝

Tomorrow: Day 7 - Complete Interpretable Pipeline
""")